In [1]:
import PyPDF2
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_ollama.llms import OllamaLLM
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate

In [2]:
pdf_paths = ["/home/bigdata/eva_1_BDA/PDF/PIMENTON_MAYO.pdf", "/home/bigdata/eva_1_BDA/PDF/recetas.pdf"] 

## Extraer info y dividirla

In [3]:
def extraer_info_pdf(pdf_path):
    #Extrae el texto de un archivo PDF usando PyPDF2.
    text_data = ""
    try:
        with open(pdf_path, "rb") as file:
            reader = PyPDF2.PdfReader(file)
            for page in reader.pages:
                text_data += page.extract_text() + " "
    except Exception as e:
        print(f"Se produjo un error al leer el PDF: {e}")
    
    return text_data


def dividir_texto(texto):
    #Divide el texto en fragmentos manejables para la indexación.
    #extrae el texto de los archivos PDF y lo divide en fragmentos de 500 caracteres, con un solapamiento de 50 caracteres,
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    return splitter.split_text(texto)


def extraer_texto_y_dividir_pdf(pdf_path):
    #Extrae el texto de un archivo PDF y lo divide en fragmentos.
    texto = extraer_info_pdf(pdf_path)
    if not texto:
        print("El texto del PDF estaba vacío")
        texto = []
    
    return dividir_texto(texto)

def extraer_y_dividir_texto_de_pdfs(pdf_paths):
    #Une los dos pdf divididos
    all_chunks = []
    for pdf_path in pdf_paths:
        chunks = extraer_texto_y_dividir_pdf(pdf_path)
        all_chunks.extend(chunks)  # Unir todos los fragmentos de los diferentes PDFs
    return all_chunks

## Crear embeddings y vectorstore

In [4]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

chunks = extraer_y_dividir_texto_de_pdfs(pdf_paths)

vectorstore = Chroma.from_texts(
        texts=chunks,
        collection_name="pdf_data",
        embedding=embeddings,
        persist_directory="./vectorstoreCastellano"
    )

/home/bigdata/miniconda3/envs/rag/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Hacer consultas a Ollama

In [5]:
llm = OllamaLLM(model="llama3.2", server_url="http://localhost:11434")

# Crear el prompt para la cadena de preguntas y respuestas
prompt = ChatPromptTemplate.from_template(
    template= "Use el contexto proporcionado en los siguientes PDFs para responder a la pregunta del usuario:"
        "\n\n{context}\n\nPregunta: {question}\nRespuesta:"
)

# Función para realizar una consulta al vector store
def realizar_consulta(vectorstore, consulta):
    retriever = vectorstore.as_retriever()
    qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, chain_type_kwargs={"prompt": prompt})
    respuesta = qa_chain.run(consulta)
    return respuesta

In [10]:
# Ejemplo de consulta pdf PIMENTON_MAYO
consulta = "Elaboración de pulpo completa"
respuesta = realizar_consulta(vectorstore, consulta)
print(f"Respuesta: {respuesta}")

Respuesta: Para elaborar el plato de Pulpo a la Gallega, sigue los siguientes pasos:

1. Retira el pulpo del fuego y reserva.
2. Para servir, coloca las patatas picadas (preferiblemente en rodajas) cubriendo toda la base del plato.
3. Corta los tentáculos en trozos de más o menos 1 cm y colócalos sobre las patatas. Añade sal al gusto.
4. Mezcla el pimentón picante y dulce en partes iguales y espolvorea por encima del pulpo.

Además, debes seguir la elaboración específica:

- En una olla con agua hirviendo, introduce el pulpo durante 5 segundos (unas 3 o 4 veces seguidas) para evitar perder la piel y que los tentáculos se ericen.
- Una vez hecho esto, déjalo en el agua hirviendo y tápalo durante 35-45 minutos a fuego medio.


In [9]:
# Ejemplo de consulta pdf recetas
consulta = "Ingredientes de cordero al horno con manzana"
respuesta = realizar_consulta(vectorstore, consulta)
print(f"Respuesta: {respuesta}")

Respuesta: Los ingredientes del "Cordero al horno con manzana" son:

* Una paletilla de corder o de unos 1,800 grs. -2 kilos
* 2 tomates maduros
* 1 cebolla
* 20 dátiles sin hueso
* 1 manzana reineta
* Media cucharada de tomillo
* Un vaso de vino blanco o coñac
* Aceite, sal y pimienta
